#### Load needed modules

In [2]:
import pandas as pd
import numpy as np
import math
import os

#### 1. Import Data

We use data from the __bitbucket__ repository of EUcalc and Data downloaded from the EUcalc Website directly. At first we import the data on population, once the historical (__pop_hist__) and the the future population (__pop_life__). Also we load the levers defined for the _Life_ scenario to be able to calculate the correct values for each variable. 

The Levers hereby indicate the ambition level for a given category, meaning the higher the lever the higher the ambition in that category. The levers go from 1-4, where for each lever future values are calculated and available on the EUcalc repository. For some categories, like population, the Lever is a decimal number, so the correct value for that category has to be interpolated with the given levers. This is done in the __calculate_new_lever__ function.

In [3]:
# historic population
pop_hist = pd.read_csv('input/eu_calc_repo_data/ots_lfs_pop.csv').drop(["iiasa_ssp2"], axis=1).rename(columns={"ots_lfs_pop_population[inhabitants]":"population"})

# get all countries listed in the data
countries = pop_hist['Country'].unique()

# population in the life scenario
pop_life = pd.read_csv("input/eu_calc_repo_data/fts_lfs_pop.csv").drop("iiasa_ssp2", axis=1).rename(columns={"fts_lfs_pop_population[inhabitants]":"population", "lever_pop":"lever"})

# Levers per Category of LIFE Scenario
levers_life = pd.read_json('input/levers_life.json').T

# Change the column name to lever for accessability
levers_life.columns = ['lever']


### 2. Process Population Data

#### 2.1 Calculate decimal levers

For some categories, like the population there is no direct lever but a decimal lever, so we need to interpolate. Therefore we give the function the data, the wanted column in that data, the lever value and the column name where the lever value can be found. Then we interpolate to get the value for the given decimal lever and return a Dataframe with the lever_value, Years and Countries. 

In [4]:
# Calculates the lever for a given column and Dataframe. Is used when lever is decimal e.g. 1.5
def calculate_new_lever(data, column, lever, lever_column='lever'):
    '''
    Calculates the lever for a given column and Dataframe. Is used when lever is decimal e.g. 1.5
    :param data: Dataframe with the data
    :param column: Column name for which the lever should be calculated
    :param lever: Lever value
    :param lever_column: Column name where the value value is found
    :return: Dataframe with the calculated lever
    '''
    # Get lower and upper lever
    lower_lever = math.floor(lever)
    upper_lever = math.ceil(lever)

    # Get the decimal part
    decimal_lever = lever % 1

    # Select the data for lower and upper lever boundary
    data_lower_lever = data[(data[lever_column] == lower_lever)].reset_index()
    data_upper_lever = data[(data[lever_column] == upper_lever)].reset_index()

    # Calculate the values according to the given lever
    column_mean_lever = data_lower_lever[column] * (1-decimal_lever) + data_upper_lever[column] * decimal_lever

    # Create a Dataframe with the new lever. Hereby we take as basis a DataFrame with only one lever selected (here the lower lever)
    # which has the same indices and columns as the newly calculated lever. Then we change the lever value and the interpolated values
    data_mean_lever = data_lower_lever.copy()
    data_mean_lever = data_mean_lever.drop(columns=['index'])
    data_mean_lever[column] = column_mean_lever
    data_mean_lever[lever_column] = lever

    return data_mean_lever

#### 2.2 Calculate Population Data for Life Scenario

For the Life scenario we have a lever of __1.5__ for the population, so we calculate the needed values with the function defined above. Furthemore we calculate the share of each age group and bring together historical and future data 

In [5]:
# Calculate population for lever 1.5 and aggregate the age groups together
pop_lever_1_5 = calculate_new_lever(pop_life, 'population', 1.5)

# Calculate the share of each age group
pop_lever_1_5['share age group'] = pop_lever_1_5.apply(lambda x: x['population'] / pop_lever_1_5[(pop_lever_1_5['Country'] == x['Country']) & (pop_lever_1_5['Years'] == x['Years'])]['population'].sum(), axis=1)

# Get total population from historic and future populations by creating a new dataframe
pop_tot = pd.concat([pop_hist, pop_lever_1_5])

# Calculate the Population of the EU
pop_tot_EU = pop_tot.groupby('Years').agg({'population':'sum'})

### 3. Process Data for each country

#### 3.1 Define needed Values
We define the needed values, as provided in the Google Docs document. Therefore we make use of a dictionary, so we can define for each wanted variable needed parameters. There are parameters used for all variables, and some depend it the Boolean __from_scenario__ is True or False. 

When the Booleans is True, the data was taken from the EUcalc Website and is available as one csv-file per country and already processed to meet the correct lever for the Life scenario (folder __country_data__). Otherwise the data is from the Bitbucket repo and processed to use the correct lever (folder __data__).


General Parameters are:
- __from_scenario__: _Boolean_; Defines if the Data for that variable comes from the scenario specific data (True) or is calculated from the data from the EUcalc repository (False)
- __filename__: _String_; Defines the filename in which the data for that variable can be found
- __unit__: _String_; The unit for that variable, is used to make the resulting DataFrame more viewable
- __per_day__: _Boolean_; Defines if the Data from EUcalc is defines as 'perDay' and has to be processed to get the annual data

Parameters when __from_scenario__ == True:
- __column__: _Boolean_; The column needed for that variable. Is used to access the correct data. If 'sum' is defined, the sum of all columns is used.

Parameters when __from_scenario__ == False:
- __lever__: _String_; Defines the name of the lever
- __age__: _Boolean_; Define if for that variable the data is available as per Age Groups Data. If yes, it is processed to get the total data.
- __data__: _Array_; Defines the columns used to calculate the needed value. Hereby the sum of all columns will be taken to calculate the needed values for the variable.

In [6]:
# All needed lever values
needed_values_dict = {
    'final energy demand': 
    {
        'from_scenario': True,
        'filename': 'effective-energy-demand',
        'unit': 'TWh',
        'per_day':False,
        'column': 'sum'
    },
    'final energy demand industry': {
        'from_scenario': True,
        'filename': 'effective-energy-demand',
        'unit': 'TWh',
        'per_day':False,
        'column': 'Industry'
    },
    'living space':
    {
        'lever':'lever_floor-intensity',
        'filename': 'fts_lfs_floor-intensity.csv',
        'filename_hist': 'ots_lfs_floor-intensity.csv',
        'from_scenario': False,
        'age': True,
        'per_day': False,
        'unit': 'm2',
        'data': ['fts_lfs_floor-intensity_space-cap[m2/cap]'] 
    },
    'floor area in commercial and public buildings':
    {
        'lever':'lever_nonres-floorarea',
        'filename': 'fts_bld_nonres-floorarea.csv',
        'filename_hist': 'ots_bld_nonres-floorarea.csv',
        'lever_value':4,
        'from_scenario': False,
        'age': False,
        'per_day': False,
        'unit': '1000m2',
        'data': ['fts_bld_nonres-floorarea_offices[1000m2]','fts_bld_nonres-floorarea_hotels[1000m2]','fts_bld_nonres-floorarea_trade[1000m2]','fts_bld_nonres-floorarea_education[1000m2]','fts_bld_nonres-floorarea_health[1000m2]', 'fts_bld_nonres-floorarea_other[1000m2]'] 
    },
    'average distance travelled':
    {   
        'lever':'lever_pkm',
        'filename': 'fts_lfs_pkm.csv',
        'filename_hist': 'ots_lfs_pkm.csv',
        'from_scenario': False,
        'age': True,
        'per_day': False,
        'unit': 'pkm',
        'data': ['fts_lfs_pkm_pkm[pkm/cap]']
        
    },
    'average distance travelled car':
    {
        'from_scenario': True,
        'filename': 'passenger-distance-per-m',
        'unit': 'pkm',
        'per_day':False,
        'column': 'Cars'
    },
    'average distance travelled plane': 
    {
        'from_scenario': True,
        'filename': 'passenger-distance-per-m',
        'unit': 'pkm',
        'per_day':False,
        'column': 'Aviation'
    },
    'transported goods':
    {
        'lever': 'lever_freight_tkm',
        'filename': 'fts_tra_freight_tkm.csv',
        'filename_hist': 'ots_tra_freight_tkm.csv',
        'from_scenario': False,
        'age': False,
        'per_day': False,
        'unit': 'km',
        'data': ['fts_tra_freight_tkm_lastmile-total-demand[bn_tkm]', 'fts_tra_freight_tkm_longdistance-total-demand[bn_tkm]']
    },
    'meat consumption':
    {
        'lever':'lever_diet',
        'filename': 'fts_lfs_diet.csv',
        'filename_hist': 'ots_lfs_diet.csv',
        'from_scenario': False,
        'age':True,
        'per_day': True,
        'unit': 'kcal',
        'data': ['fts_lfs_diet_bov[kcal/cap/day]','fts_lfs_diet_sheep[kcal/cap/day]','fts_lfs_diet_pigs[kcal/cap/day]','fts_lfs_diet_poultry[kcal/cap/day]','fts_lfs_diet_oth-animals[kcal/cap/day]']
    },
    'food waste':
    {
        'lever':'lever_fwaste',
        'filename': 'fts_lfs_fwaste.csv',
        'filename_hist': 'ots_lfs_fwaste.csv',
        'from_scenario': False,
        'age': True,
        'per_day': True,
        'unit': 'kcal',
        'data': ['fts_lfs_fwaste_wine[kcal/cap/day]','fts_lfs_fwaste_beer[kcal/cap/day]','fts_lfs_fwaste_bev-fer[kcal/cap/day]','fts_lfs_fwaste_bev-alc[kcal/cap/day]','fts_lfs_fwaste_cereals[kcal/cap/day]','fts_lfs_fwaste_fruits[kcal/cap/day]','fts_lfs_fwaste_oilcrops[kcal/cap/day]','fts_lfs_fwaste_pulses[kcal/cap/day]','fts_lfs_fwaste_starch[kcal/cap/day]','fts_lfs_fwaste_coffee[kcal/cap/day]','fts_lfs_fwaste_stm[kcal/cap/day]','fts_lfs_fwaste_sugar[kcal/cap/day]','fts_lfs_fwaste_sweet[kcal/cap/day]','fts_lfs_fwaste_voil[kcal/cap/day]','fts_lfs_fwaste_veg[kcal/cap/day]','fts_lfs_fwaste_dfish[kcal/cap/day]','fts_lfs_fwaste_ffish[kcal/cap/day]','fts_lfs_fwaste_pfish[kcal/cap/day]','fts_lfs_fwaste_seafood[kcal/cap/day]','fts_lfs_fwaste_oth-aq-animals[kcal/cap/day]','fts_lfs_fwaste_egg[kcal/cap/day]','fts_lfs_fwaste_milk[kcal/cap/day]','fts_lfs_fwaste_offal[kcal/cap/day]','fts_lfs_fwaste_bov[kcal/cap/day]','fts_lfs_fwaste_sheep[kcal/cap/day]','fts_lfs_fwaste_pigs[kcal/cap/day]','fts_lfs_fwaste_poultry[kcal/cap/day]','fts_lfs_fwaste_oth-animals[kcal/cap/day]','fts_lfs_fwaste_afats[kcal/cap/day]','fts_lfs_fwaste_rice[kcal/cap/day]']
    },
    'cement production':
    {
        'from_scenario': True,
        'filename': 'material-production',
        'unit': 'Mt',
        'per_day':False,
        'column': 'Cement'
    },
    'steel production':
    {
        'from_scenario': True,
        'filename': 'material-production',
        'unit': 'Mt',
        'per_day':False,
        'column': 'Steel'
    }
}


#### 3.2 Process Repository Data

For the Data from the EUcalc repository we need to process it, so we get the correct data for the lever defined in the Life scenario. We do that in the next step, and the create a csv-file per country containing the data of each variable.

In [7]:
# Gets the lever_value from the levers dictonary defined for the Life scenario
# Then calculates the new values. Depending if its a decimal or not, it used the calculate_new_lever function
def get_lever_values(data, lever, columns, age, variable, country, variable_parameters):
    '''
    Gets the lever_value from the levers dictonary defined for the Life scenario
    Then calculates the new values. Depending if its a decimal or not, it used the calculate_new_lever function
    :param data: Dataframe with the data
    :param lever: Lever name
    :param columns: Columns for which the lever should be calculated
    :param age: Boolean if the data is available in age groups
    :param variable: Variable name
    :param country: Country name
    :param variable_parameters: Variable parameters
    :return: Dataframe with the calculated lever
    '''
    # Get the lever value for life
    if 'lever_value' in variable_parameters.keys():
        lever_value = variable_parameters['lever_value']
    else:
        if lever not in levers_life.index:
            return None
        
        # Get the lever defined for the life scenario
        lever_value = levers_life.at[lever, 'lever']

    # If the data is available in age groups, get the correct value for all age groups
    if age:
       # Calculated the weighted aggregration
       pop_country = pop_lever_1_5[pop_lever_1_5['Country'] == country]
       share_age_group = pop_country[['Age', 'share age group']]
       data = data[['Country', 'Years'] + [lever] +  ['Age'] + columns]
       data['share age'] = data.apply(lambda x: share_age_group[share_age_group['Age'] == x['Age']]['share age group'].values[0], axis=1) 
       data[columns] = data[columns].multiply(data['share age'], axis='index')
       data = data.drop(columns=['share age'])
       data = data.groupby(['Country', 'Years', f'{lever}'], as_index =False).sum()

    try:
        # Check if Lever is a clean value, than we can directly get the data
        if lever_value % 1 == 0:
            data_lever = data[(data[lever] == lever_value) & (data['Country'] == country)]
            data_lever[variable] = data_lever[columns].sum(axis=1, numeric_only=True)
            data_lever = data_lever[['Country', 'Years'] + [lever] + [variable]]
            return data_lever
    
        # Otherwise calculate the data based on the lever 
        else:
            data = data[data['Country'] == country]
            data_lever = data[['Years', 'Country']].copy()
            for column in columns:
                data_column_lever = calculate_new_lever(data, column, lever_value, lever_column=lever)
                data_lever[column] = data_column_lever[column]
            data_lever[variable] = data_lever[columns].sum(axis=1, numeric_only=True)
            data_lever = data_lever[['Country', 'Years'] + [lever] + [variable]]
            return data_lever
    except Exception as e:
        print(e)
        return None

In [8]:
# Iterate over all countries to get a csv-file per country containing the wanted variables
# WARNING, Takes ca. 12 minutes. Can be skipped if all data is already in the output/country_data folder.
for country in countries:

    # Needed counter to take the first Data as initial Dataframe, to be able to concentate
    counter = 0

    # Iterate over all needed values defined in the dictionary above
    for key in needed_values_dict.keys():

        # Get the parameters of that variable
        variable_parameters = needed_values_dict[key]

        # Check where the Data comes from and only process the variables where the Data comes from the Repository
        if variable_parameters['from_scenario'] == False:

            # Get the filename
            filename = variable_parameters['filename']

            # Load the data
            data_file = pd.read_csv(f'input/eu_calc_repo_data/{filename}')


            # Check for the counter and if its the first variable, use it as initial DataFrame
            if counter == 0:
                counter = 1

                # Calculate the values of the variable
                data_lever_initial = get_lever_values(data_file, variable_parameters['lever'], variable_parameters['data'], variable_parameters['age'], key, country, variable_parameters)
            else:
                # Calculate the values of the variable
                data_lever = get_lever_values(data_file, variable_parameters['lever'], variable_parameters['data'], variable_parameters['age'], key, country, variable_parameters)
                
                # Check if the Results are valid
                if data_lever is not None:

                    # Merge the Initial and new Data together
                    data_lever_initial = pd.merge(
                        left=data_lever_initial,
                        on=['Years', 'Country'],
                        right=data_lever,
                        how='left'
                    )
        else:
            continue
    
    # Export Data as csv
    data_lever_initial.to_csv(f'output/country_data/{country}_life_data.csv')


C:\Users\yanni\AppData\Local\Temp\ipykernel_5792\1260999407.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['share age'] = data.apply(lambda x: share_age_group[share_age_group['Age'] == x['Age']]['share age group'].values[0], axis=1)
C:\Users\yanni\AppData\Local\Temp\ipykernel_5792\1260999407.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[columns] = data[columns].multiply(data['share age'], axis='index')
C:\Users\yanni\AppData\Local\Temp\ipykernel_5792\1260999407.py:30: SettingWithCopyW

KeyboardInterrupt: 

#### 3.3 Create Long DataFrame

Now we put all needed variables together into one DataFrame with the columns 'Year', 'Country', 'Variable name', 'Variable total', 'Variable per capita'. Therefore we use the csv-files created in the step before, as well as the perCountry files for the variables with __from_scenario__ == True. 

In [14]:
# Define a initial Dataframe 
eu_calc_long_dataframe = pd.DataFrame([], columns=['Year', 'Country', 'Variable name', 'Variable total', 'Variable per capita'])

# Iterate over the countries
for country in countries:

    # Load the csv-file created in the step before
    data_country = pd.read_csv(f'output/country_data/{country}_life_data.csv')

    # Get the Population for the country
    pop_country_abs = pop_tot[pop_tot['Country'] == country].groupby(['Years'], as_index=False).agg({'population':'sum'})
    
    # Get the years
    years = pop_country_abs['Years'].unique()

    # Iterate over the needed values
    for key in needed_values_dict.keys():

        # Get the parameters for that variable
        variable_params = needed_values_dict[key]

        # Check for the from_scenario Bool
        if variable_params['from_scenario'] == False:

            # load historic data 
            data_variable_hist =  pd.read_csv(f'input/eu_calc_repo_data/{variable_params["filename_hist"]}')

            # Select country specific data, where only years below 2020 are accounted
            data_variable_hist = data_variable_hist[(data_variable_hist['Country'] == country) & (data_variable_hist['Years'] < 2020)]

            # rename columns so they are named in the way as they're defined in the needed_values_dict
            columns = [x.replace('fts_', '') if x.replace('fts_', '') in data_variable_hist.columns else x.replace('fts', 'ots') for x in variable_params["data"]]
            
            # sum up all the needed data defined in the needed_values_dict
            data_variable_hist = data_variable_hist[columns].sum(axis=1, numeric_only=True)
 
            # If False, the data can be directly accessed in the country specific csv file created before.
            if key in data_country.columns:
                data_variable = data_country[key]
                data_variable = pd.concat([data_variable_hist, data_variable])

                # Check if data needs to be processed to annual data
                if variable_params['per_day']:
                    data_variable = data_variable * 365

                # Check if the data is age specific. If yes, it is also always given as perCapita
                if variable_params['age']:
                    
                    # Calculate the absolute and per Capita data
                    data_variable_capita = data_variable.values
                    data_variable_abs = data_variable.values * pop_country_abs['population']

                else:
                    # Calculate the absolute and per Capita data
                    data_variable_capita = data_variable.values / pop_country_abs['population']
                    data_variable_abs = data_variable.values
            else:
                print('Error, key not in Life data')
                print(variable_params["lever"])
            
        else:
            # If from_scenario == True, load the correct csv-file
            data_variable = pd.read_csv(f'input/country_data/{variable_params["filename"]}_{country}.csv', index_col=[0], sep=";", decimal=',')
            
            # Select only the needed years
            data_variable = data_variable.loc[years]

            # Check if one column or the sum is wanted
            if variable_params['column'] == 'sum':
                data_variable = data_variable.sum(axis=1)
                
                # Calculate the absolute and per Capita data
                data_variable_abs = data_variable.values
                data_variable_capita = data_variable.values / pop_country_abs['population']
            else:
                data_variable = data_variable[variable_params['column']]
                
                # Calculate the absolute and per Capita data
                data_variable_abs = data_variable.values
                data_variable_capita = data_variable.values / pop_country_abs['population']
        
        # Create a Dataframe for the Country and the Variable
        data_variable_country_df = pd.DataFrame([], columns=['Year', 'Country', 'Variable total', 'Variable per capita'])
        data_variable_country_df['Year'] = years
        data_variable_country_df['Country'] = country
        data_variable_country_df['Variable name'] = f'{key}'
        data_variable_country_df['Variable total'] = data_variable_abs
        data_variable_country_df['Unit total'] = variable_params["unit"]
        data_variable_country_df['Variable per capita'] = data_variable_capita
        data_variable_country_df['Unit per capita'] = f'{variable_params["unit"]} / capita'

        # Concat all variables together to a long Dataframe
        eu_calc_long_dataframe = pd.concat([eu_calc_long_dataframe, data_variable_country_df])

# Calculate the EU-Values
# Calculate the EU-Values
for variable in eu_calc_long_dataframe['Variable name'].unique():
    
    # Get Sum for that variable (Absolute Value)
    eu_data = eu_calc_long_dataframe[eu_calc_long_dataframe['Variable name'] == variable].groupby(['Year'], as_index=False).agg({'Variable total':'sum', 'Unit total':'first', 'Unit per capita':'first'})
    
    # Define Variable Name and Country
    eu_data['Variable name'] = variable
    eu_data['Country'] = 'EU'

    # Calculate Per Capita Value
    years = eu_data['Year'].unique()
    pop_eu = pop_tot_EU.loc[years]
    eu_data['Variable per capita'] = eu_data['Variable total'].values / pop_eu['population'].values

    # Concat all variables together to a long Dataframe
    eu_calc_long_dataframe = pd.concat([eu_calc_long_dataframe, eu_data])

# Map Historical Bool to data
eu_calc_long_dataframe['Historical'] = 1
eu_calc_long_dataframe[eu_calc_long_dataframe['Year'] > 2015]['Historical'] = 1

# Reindex the Dataframe
eu_calc_long_dataframe = eu_calc_long_dataframe.set_index(['Year', 'Country'])

# Export as CSV and Excel
eu_calc_long_dataframe.to_csv(f'output/variables_countries.csv')
eu_calc_long_dataframe.to_excel(f'output/variables_countries.xlsx')
    
    

C:\Users\yanni\AppData\Local\Temp\ipykernel_5792\1654709884.py:93: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  eu_calc_long_dataframe = pd.concat([eu_calc_long_dataframe, data_variable_country_df])
C:\Users\yanni\AppData\Local\Temp\ipykernel_5792\1654709884.py:116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eu_calc_long_dataframe[eu_calc_long_dataframe['Year'] > 2015]['Historical'] = 1
